In [8]:
import numpy as np

np.random.seed(0)

In [9]:
def dijkstra(grid, start, goal_row):
    rows, cols = grid.shape
    visitados = set()
    stack = [start]

    while stack:
        x, y = stack.pop()
        if (x, y) in visitados:
            continue
        visitados.add((x, y))

        # chegou em qualquer célula da primeira linha
        if x == goal_row:
            return 1

        # vizinhos (cima, baixo, esquerda, direita)
        for dx, dy in [(1,0), (-1,0), (0,1), (0,-1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < rows and 0 <= ny < cols and grid[nx, ny] == 0:
                stack.append((nx, ny))

    return 0

In [10]:

def generate_datasets(samples, size, prob):
    X, Y = [], []
    for _ in range(samples):
        grid = np.random.choice([0,1], (size, size), p=[1-prob, prob])
        start = (size - 1, size//2)
        grid[start] = 0
        labels = 2  # duas classes: possível (1) ou não (0)

        X.append(grid)
        Y.append(dijkstra(grid, start, 0))  # retorna 0 ou 1

    X = np.array(X)
    Y = np.array(Y)
    one_hotted_Y = np.eye(labels)[Y] 

    # ⚡ Achata cada grid 2D em vetor 1D
    X_flat = X.reshape(X.shape[0], -1)  # shape -> (samples, size*size)

    return X, X_flat, Y, one_hotted_Y


In [13]:
def init_parms(samples, size, prob):
    X_map, X, Y, y_true = generate_datasets(samples=samples, size=size, prob=prob)

    input_dim = X.shape[1]          # features
    hidden_dim = size               # tamanho da camada oculta
    output_dim = y_true.shape[1]    # número de classes (ex: 2)

    W1 = np.random.rand(input_dim, hidden_dim) 
    b1 = np.zeros((1, hidden_dim)) 
    W2 = np.random.rand(hidden_dim, output_dim) 
    b2 = np.zeros((1, output_dim)) 

    return X_map, X, Y, y_true, W1, b1, W2, b2

    
def forward_prop(X, W1, b1, W2, b2):
    Z1 = X.dot(W1) + b1
    A1 = ReLU(Z1)
    Z2 = A1.dot(W2) + b2
    y_pred = softmax(Z2)
    return Z1, A1, Z2, y_pred 

def cross_entropy(y_true, y_pred):
    eps = 1e-9
    return -np.mean(np.sum(y_true * np.log(y_pred + eps),   axis=1))
    
def backward_prop(Z1, A1, Z2, y_pred, W1, W2, X, y_true):
    N = X.shape[0]

    # Saída
    dZ2 = y_pred - y_true                # (N, output_dim)
    dW2 = np.dot(A1.T, dZ2) / N
    db2 = np.mean(dZ2, axis=0, keepdims=True)

    # Oculta
    d_hidden = np.dot(dZ2, W2.T) * ReLU_derivative(Z1)
    dW1 = np.dot(X.T, d_hidden) / N
    db1 = np.mean(d_hidden, axis=0, keepdims=True)

    return dW1, db1, dW2, db2



def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 -= alpha * dW1
    b1 -= alpha * db1   
    W2 -= alpha * dW2
    b2 -= alpha * db2
    return W1, b1, W2, b2
    

def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=1, keepdims=True))
    return expZ / np.sum(expZ, axis=1, keepdims=True)
    
def ReLU(z):
    return np.maximum(0, z)

def ReLU_derivative(z):
    return (z > 0).astype(float)


In [7]:
def get_predictions(y_pred):
    return np.argmax(y_pred, 0)

def get_predictions(y_pred):
    return np.argmax(y_pred, axis=1) 

def test_model(X_test, X_map, Y_test, W1, b1, W2, b2, samples):
    # Forward pass
    Z1 = np.dot(X_test, W1) + b1
    A1 = np.maximum(0, Z1)               # ReLU
    Z2 = np.dot(A1, W2) + b2
    expZ = np.exp(Z2 - np.max(Z2, axis=1, keepdims=True))
    y_pred = expZ / np.sum(expZ, axis=1, keepdims=True)  # softmax

    # Classes previstas
    predictions = np.argmax(y_pred, axis=1)

    # Acurácia
    accuracy = np.mean(predictions == Y_test)

    # Mostrar alguns exemplos
    print("Exemplos de teste:")
    for i in range(samples):
        if i % 100 == 0:

            print(f"Exemplo {i}:")
            print(f"  Mapa: \n{X_map[i]}")
            print(f"  Saída real: {Y_test[i]}")
            print(f"  Saída prevista: {predictions[i]}")
            print(f"  Probabilidades: {y_pred[i]}")
            print("-"*30)

    print(f"\nAcurácia total: {accuracy*100:.2f}%")

    return y_pred, predictions, accuracy


def gradient_descent(samples, size, prob, alpha):
    X_map, X, Y, y_true, W1, b1, W2, b2 = init_parms(samples, size, prob)
    for i in range(samples):
        Z1, A1, Z2, y_pred = forward_prop(X, W1, b1, W2, b2)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, y_pred, W1, W2, X, y_true)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        print(f"Iteration: {i}")
        y_pred_test, predictions, accuracy = test_model(X, X_map, Y, W1, b1, W2, b2, samples)
        print(f"Acurácia no dataset de treino: {accuracy*100:.2f}%\n")
    return W1, b1, W2, b2, X, Y, X_map

In [14]:
W1, b1, W2, b2, X, Y, X_map = gradient_descent(samples=300, size=4, prob=0.35, alpha=0.12)

Iteration: 0
Exemplos de teste:
Exemplo 0:
  Mapa: 
[[0 0 0 1]
 [1 1 1 1]
 [0 1 1 0]
 [1 0 0 1]]
  Saída real: 0
  Saída prevista: 1
  Probabilidades: [0.0743045 0.9256955]
------------------------------
Exemplo 100:
  Mapa: 
[[1 1 1 0]
 [1 1 1 1]
 [0 0 0 0]
 [1 0 0 1]]
  Saída real: 0
  Saída prevista: 1
  Probabilidades: [0.09691096 0.90308904]
------------------------------
Exemplo 200:
  Mapa: 
[[0 0 0 1]
 [0 0 0 0]
 [0 1 0 1]
 [0 0 0 1]]
  Saída real: 1
  Saída prevista: 1
  Probabilidades: [0.2557897 0.7442103]
------------------------------

Acurácia total: 68.67%
Acurácia no dataset de treino: 68.67%

Iteration: 1
Exemplos de teste:
Exemplo 0:
  Mapa: 
[[0 0 0 1]
 [1 1 1 1]
 [0 1 1 0]
 [1 0 0 1]]
  Saída real: 0
  Saída prevista: 0
  Probabilidades: [0.56335198 0.43664802]
------------------------------
Exemplo 100:
  Mapa: 
[[1 1 1 0]
 [1 1 1 1]
 [0 0 0 0]
 [1 0 0 1]]
  Saída real: 0
  Saída prevista: 0
  Probabilidades: [0.59008532 0.40991468]
------------------------------
E